# Using queries to understand AiiDA database contents #1
## General information on Computers, Groups, Nodes and node connectivity
Time: 2 mins

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()
from aiida.orm import load_node, Node, Group, Computer, User
from aiida.orm.querybuilder import QueryBuilder
from IPython.display import Image

from notebook_helpers import generate_query_graph

#### How many nodes/groups/computers/users are stored in my database?

In [ ]:
print "My database contains:"
for cls in (User, Computer, Group, Node):
    qb = QueryBuilder()
    qb.append(cls)
    count = qb.count()
    print "{:>5}  {}s".format(count, cls.__name__)

#### How many of each subclass of Group/Node are stored in my database

In [ ]:
for cls in (Node, Group):
    print '\n', 'Subclasses of {}:'.format(cls.__name__)
    qb1 = QueryBuilder()
    qb1.append(cls, project='type')
    distinct_types, = zip(*qb1.distinct().all()) # Getting all distinct types
    # Iterating through distinct types:
    for dtype in sorted(distinct_types):
        qb2 = QueryBuilder()
        qb2.append(cls, filters={'type':dtype})
        subcls_count = qb2.count()
        print '   {:<15} | {:<4}'.format(dtype.strip('.').split('.')[-1] or "N/A", subcls_count)

#### What is the connectivity of my database? How many links? How many distinct paths?

In [ ]:
# Here I query the number of links:
qb1 = QueryBuilder()
qb1.append(Node, tag='n1')
qb1.append(Node, output_of='n1')
link_count = qb1.count()
print '\nThe number of links in my database is: {}'.format(link_count)

generate_query_graph(qb1.get_json_compatible_queryhelp(), 'query-statistics-1.png')
Image(filename='query-statistics-1.png')


In [ ]:
# Here I query the number of distinct paths:
qb2 = QueryBuilder()
qb2.append(Node, tag='n1')
qb2.append(Node, descendant_of='n1')
path_count = qb2.count()
print '\nThe number of distinct paths in my database is: {}'.format(path_count)

generate_query_graph(qb2.get_json_compatible_queryhelp(), 'query-statistics-2.png')
Image(filename='query-statistics-2.png')